In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Mounted at /content/gdrive


In [ ]:
%tensorflow_version 1.x 

TensorFlow 1.x selected.


In [ ]:
# 텐서플로우 허브로부터 다양한 사전 훈련된 모델(Pre-tained Model)들을 사용
# 사전 훈련된 모델로부터 ELMo 표현을 사용
! pip install tensorflow-hub

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
from keras import backend as K
import urllib.request
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [ ]:
  import pandas as pd
  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn.model_selection import train_test_split
  from tensorflow.keras.utils import to_categorical
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense
  from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
  from tensorflow.keras.models import load_model
  from sklearn.metrics.pairwise import cosine_similarity
  import nltk
  from nltk.corpus import stopwords
  from nltk.stem import WordNetLemmatizer
  import re
  from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# 텐서플로우 허브로부터 ELMo를 다운로드
elmo = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
# 텐서플로우 허브로부터 ELMo를 다운로드

sess = tf.Session()
K.set_session(sess)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

In [ ]:
'''데이터 로드'''
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding_hyun/data/dataset_210626_215600.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

X_data = df['lemmatizated']

In [ ]:
df['category'] = df['name'].astype('category')
df['category'].cat.categories = [i for i in range(0, len(df['category'].cat.categories))]
df['category']

0         3
1         3
2         3
3         3
4         3
         ..
74774    48
74775    48
74776    48
74777    48
74778    48
Name: category, Length: 74779, dtype: category
Categories (89, int64): [0, 1, 2, 3, ..., 85, 86, 87, 88]

In [ ]:
# 라벨을 원핫 벡터로 만들기
y_data = df['category']
y_data = to_categorical(y_data)

In [ ]:
# 전체 데이터 개수의 80%와 20% 개수 확인
print(len(X_data))
n_of_train = int(len(X_data) * 0.8)
n_of_test = int(len(X_data) - n_of_train)
print(n_of_train)
print(n_of_test)

74779
59823
14956


In [ ]:
# train, test split
X_train = np.asarray(X_data[:n_of_train]) #X_data 데이터 중에서 앞의 4457개의 데이터만 저장
y_train = np.asarray(y_data[:n_of_train]) #y_data 데이터 중에서 앞의 4457개의 데이터만 저장
X_test = np.asarray(X_data[n_of_train:]) #X_data 데이터 중에서 뒤의 1115개의 데이터만 저장
y_test = np.asarray(y_data[n_of_train:]) #y_data 데이터 중에서 뒤의 1115개의 데이터만 저장

In [ ]:
def ELMoEmbedding(x):
    return elmo(tf.squeeze(tf.cast(x, tf.string)), as_dict=True, signature="default")["default"]
# 데이터의 이동이 케라스 → 텐서플로우 → 케라스가 되도록 하는 함수

### Model

In [ ]:
from tensorflow.keras.utils import multi_gpu_model

In [ ]:
from keras.models import Model
from keras.layers import Dense, Lambda, Input

input_text = Input(shape=(1,), dtype=tf.string)
embedding_layer = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text)
hidden_layer = Dense(256, activation='relu')(embedding_layer)
output_layer = Dense(89, activation='softmax')(hidden_layer)
model = Model(inputs=[input_text], outputs=output_layer)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
history = model.fit(X_train, y_train, epochs=1, batch_size=32)

Epoch 1/1
48960/59823 [=======================>......] - ETA: 4:48 - loss: 4.0867 - accuracy: 0.0410

ResourceExhaustedError: ignored

In [ ]:
model.save('/content/gdrive/MyDrive/Colab Notebooks/deeplearning_NLP/perfume/word embedding/model/elmo_embedding')

In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))